In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [4]:
# 비디오 파일 이름
input_file = "D:\\Pitch\\pitch_videos\\hyojong_front.mp4" # 여기에 비디오 경로 넣으면 됩니다!

# 비디오 파일의 경우 이것을 사용하세요:
cap = cv2.VideoCapture(input_file)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [5]:
# 출력 파일 경로 및 이름 설정
output_file = os.path.splitext(input_file)[0] + "_dotted.mp4"
out = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

In [6]:
# 팔꿈치 관절 좌표화 코드
# left_elbow_coordinates = [] # 좌완투수
right_elbow_coordinates = [] # 우완투수, 위 리스트와 둘 중 하나는 주석처리해야 합니다.

with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("비디오 파일을 처리하는데 실패했습니다.")
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        
        image_height, image_width, _ = image.shape

        if results.pose_landmarks:
            # left_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            # cv2.circle(image, (int(left_elbow.x * image_width), int(left_elbow.y * image_height)), 5, (0, 255, 0), -1)
            # left_elbow_coordinates.append((left_elbow.x, left_elbow.y))

            right_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            cv2.circle(image, (int(right_elbow.x * image_width), int(right_elbow.y * image_height)), 5, (0, 0, 255), -1)
            right_elbow_coordinates.append((right_elbow.x, right_elbow.y))

        out.write(image)

        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
out.release()  # 출력 파일 닫기
cv2.destroyAllWindows()

비디오 파일을 처리하는데 실패했습니다.


In [8]:
# 팔꿈치 좌표 출력
for i, coord in enumerate(right_elbow_coordinates):
    print(f"Frame {i+1}: x = {coord[0] * image_width}, y = {coord[1] * image_height}")

Frame 1: x = 675.5080413818359, y = 376.34109020233154
Frame 2: x = 675.6867980957031, y = 376.3881254196167
Frame 3: x = 675.2035522460938, y = 376.3894557952881
Frame 4: x = 675.1422119140625, y = 376.388897895813
Frame 5: x = 675.7374572753906, y = 376.67407035827637
Frame 6: x = 677.4628448486328, y = 378.0892038345337
Frame 7: x = 679.591064453125, y = 379.37378883361816
Frame 8: x = 681.8024444580078, y = 380.0267028808594
Frame 9: x = 682.3105621337891, y = 380.47478199005127
Frame 10: x = 684.6803283691406, y = 380.54863929748535
Frame 11: x = 689.5814514160156, y = 381.2479019165039
Frame 12: x = 698.1575012207031, y = 381.7585515975952
Frame 13: x = 694.4061279296875, y = 381.294207572937
Frame 14: x = 695.9377288818359, y = 381.1864900588989
Frame 15: x = 699.9070739746094, y = 381.037015914917
Frame 16: x = 705.3431701660156, y = 381.3139486312866
Frame 17: x = 705.7598876953125, y = 380.62824726104736
Frame 18: x = 703.2701873779297, y = 379.4729232788086
Frame 19: x = 701

In [10]:
# 좌표를 데이터프레임에 저장
import pandas as pd
df = pd.DataFrame(right_elbow_coordinates, columns=["x", "y"])
df.head()

,x,y
0,0.527741,0.522696
1,0.527880,0.522761
2,0.527503,0.522763
3,0.527455,0.522762
4,0.527920,0.523158


In [19]:
output_dir = "D:/Pitch/coordinates"
output_filename = os.path.basename(os.path.splitext(input_file)[0]) + "_elbow_coordinates.csv"
output_file_path = os.path.join(output_dir, output_filename)

df.to_csv(output_file_path, index=False)

In [8]:
import cv2

cap = cv2.VideoCapture(output_file)

if (cap.isOpened()== False): 
    print("비디오를 열 수 없습니다.")
  
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        cv2.imshow('Frame', frame)
        # q 키를 눌러 비디오 재생을 중지합니다.
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else: 
        break

cap.release()
cv2.destroyAllWindows()

# q 누르면 비디오 중지됩니다!!! 제발!!!